In [35]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드

api_key = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)
# print(api_key)

In [29]:
from PIL import Image # Make sure you have 'Pillow' installed (pip install Pillow)
from io import BytesIO

# Assume 'image' is your file path string as before
image_path = r'C:\Users\Admin\kpmg_future_lab\llm_basic\geminai_api_ex\output.png'

try:
    # 1. Load the image from the path into a PIL Image object
    #    This 'image' variable will now be an actual image object
    #    If you're getting the image from a GenAI response,
    #    the 'image = Image.open(BytesIO(part.inline_data.data))' line
    #    from your previous code would fulfill this step.
    image_object = Image.open(image_path)

    # Now, 'image_object' is a PIL Image, which has the .copy() method
    if image_object is not None:
        temp_processed_image = image_object.copy() # 원본을 복사해서 사용

        # 이미지 보여주기 (선택 사항)
        # temp_processed_image.show()

        # 이미지 저장하기
        save_path_simulation = "saved_original_poodle.png"
        temp_processed_image.save(save_path_simulation)
        print(f"예시 이미지가 '{save_path_simulation}' 파일로 저장되었습니다.")
    else:
        print("이미지를 로드할 수 없어 저장 예시를 실행할 수 없습니다.")

except FileNotFoundError:
    print(f"오류: '{image_path}' 경로에서 이미지를 찾을 수 없습니다. 경로를 확인해주세요.")
except Exception as e:
    print(f"이미지 처리 또는 저장 중 오류 발생: {e}")

예시 이미지가 'saved_original_poodle.png' 파일로 저장되었습니다.


In [38]:
import PIL.Image
from google import genai
from google.genai import types
from PIL import Image
import base64
from io import BytesIO
import os
from datetime import datetime

# 이미지를 base64로 변환하는 함수
def image_to_base64(image):
    buffer = BytesIO()
    image.save(buffer, format='JPEG')
    img_data = buffer.getvalue()
    return base64.b64encode(img_data).decode()

# 클라이언트 설정
client = genai.Client(api_key=GEMINI_API_KEY)

# 이미지 로드 및 변환
image = PIL.Image.open('./lady.jpg')
image_b64 = image_to_base64(image)

text_input = ('Hi, This is a picture of me. '
              'Can you add a llama next to me?')

# API 호출
response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=[
        {
            "role": "user", 
            "parts": [
                {"text": text_input},
                {"inline_data": {
                    "mime_type": "image/jpeg",
                    "data": image_b64
                }}
            ]
        }
    ],
    config=types.GenerateContentConfig(
        response_modalities=['TEXT', 'IMAGE']
    )
)

In [39]:
# 결과 처리 및 저장
output_dir = './generated_images'
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for i, part in enumerate(response.candidates[0].content.parts):
    if part.text is not None:
        print(f"Generated text: {part.text}")
        
        # 텍스트도 저장하고 싶다면
        with open(f'{output_dir}/response_text_{timestamp}.txt', 'w', encoding='utf-8') as f:
            f.write(part.text)
            
    elif part.inline_data is not None:
        # 이미지 데이터를 PIL Image로 변환
        image_data = part.inline_data.data
        
        # base64 문자열인 경우 디코딩
        if isinstance(image_data, str):
            image_bytes = base64.b64decode(image_data)
        else:
            image_bytes = image_data
            
        # PIL Image로 변환
        generated_image = Image.open(BytesIO(image_bytes))
        
        # 이미지 저장
        output_filename = f'{output_dir}/generated_image_{timestamp}_{i}.jpg'
        generated_image.save(output_filename, 'JPEG', quality=95)
        
        print(f"Image saved: {output_filename}")
        
        # 이미지 표시 (선택사항)
        generated_image.show()

print(f"All outputs saved in: {output_dir}")

Generated text: I will add a fluffy, white llama standing calmly next to you in the grassy field, ensuring it is in proportion to your size and fits naturally into the existing outdoor scene with the distant hill and lighthouse.

Image saved: ./generated_images/generated_image_20250528_152419_1.jpg
All outputs saved in: ./generated_images


# 유튜브

In [40]:
response = client.models.generate_content(
    model='models/gemini-2.0-flash',
    contents=types.Content(
        parts=[
            types.Part(
                file_data=types.FileData(file_uri='https://www.youtube.com/watch?v=9hE5-98ZeCg')
            ),
            types.Part(text='Please summarize the video in 3 sentences.')
        ]
    )
)

In [43]:
print(response.text)

Sure! This video is a demonstration of Gemini 2.0's ability to interact with Google's AI Studio. In the video, a live stream is cast to the AI studio, and the narrator asks the AI to read a document that is being streamed. The narrator then asks the AI to summarize the demonstration and read the video's ending card.
